In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import os
import itertools
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
#df0 = pd.read_csv("Final_Data.csv")
df0 = pd.read_csv("delhi_airport_data.txt")

In [ ]:
df0.describe()

,tmpc,dwpc,relh,feel,sped,p01m,vsby
count,281705.000000,281380.000000,278559.000000,272100.000000,282653.000000,130483.0,283994.000000
mean,25.682234,15.229531,58.421721,81.309749,5.629358,0.0,1.754066
std,8.374250,7.098401,23.799553,17.648865,4.234063,0.0,1.146923
min,0.000000,-8.000000,5.210000,28.530000,0.000000,0.0,-99.000000
25%,20.000000,10.000000,39.270000,68.000000,3.450000,0.0,1.120000
50%,27.000000,14.000000,59.010000,82.900000,5.750000,0.0,1.740000
75%,32.000000,22.000000,78.040000,95.080000,8.050000,0.0,2.240000
max,56.000000,37.000000,100.000000,138.620000,147.200000,0.0,80.000000


In [ ]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286931 entries, 0 to 286930
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   station  286931 non-null  object 
 1   valid    286931 non-null  object 
 2   tmpc     281705 non-null  float64
 3   dwpc     281380 non-null  float64
 4   relh     278559 non-null  float64
 5   feel     272100 non-null  float64
 6   sped     282653 non-null  float64
 7   p01m     130483 non-null  float64
 8   vsby     283994 non-null  float64
dtypes: float64(7), object(2)
memory usage: 19.7+ MB


In [ ]:
#converting columns to float type
for col in df1.columns:
    if ((col != "station") & (col != "valid")):
        df0[col] = df0[col].astype(float)

#converting to IST
df0['valid'] = pd.to_datetime(df0['valid']) + timedelta(hours=5, minutes=30)
df0['vsby'] = df0['vsby']*1.60934 # Conversion from miles to km
df0[df0['vsby']<0] = math.nan #handling invalid values

In [ ]:
#handling missing values using linear interpolation
def interpolate(dfi):
    #dfi = dfi.drop(dfi.loc[dfi.index.year<2012].index)
    dfi = dfi.interpolate(method='linear')
    return dfi

df1 = interpolate(df0)

In [ ]:
#extracting the winter months
timestamp = df1['valid']
df2 = df1.set_index('valid')

timestamp = pd.DatetimeIndex(timestamp)

df2 = df2.reindex(timestamp)
cols = df2.columns.difference(['vsby'])
df2[cols] = df2[cols].ffill()
df2['day'] = timestamp.day
df2['year'] = timestamp.year
df2['month'] = timestamp.month
df2['time'] = timestamp.time

#selecting the required columns
df3 = df2[['year','month','day','time','vsby','tmpc','dwpc','relh','sped']]

#selecting nov-march months
df4 = df3[df3.month.isin([1,2,3,11,12])]

In [ ]:
df4.head(40)

,year,month,day,time,vsby,tmpc,dwpc,relh,sped
valid,,,,,,,,,
1999-11-01 06:00:00,1999.0,11.0,1.0,06:00:00,1.609340,21.0,15.0,68.550,4.60
1999-11-01 07:00:00,1999.0,11.0,1.0,07:00:00,1.609340,23.0,15.0,60.670,6.90
1999-11-01 08:00:00,1999.0,11.0,1.0,08:00:00,1.609340,25.0,16.0,57.360,5.75
1999-11-01 09:00:00,1999.0,11.0,1.0,09:00:00,1.609340,28.0,15.0,45.070,6.90
1999-11-01 10:00:00,1999.0,11.0,1.0,10:00:00,1.609340,30.0,14.0,37.620,9.20
1999-11-01 11:00:00,1999.0,11.0,1.0,11:00:00,2.414010,31.0,14.0,35.530,4.60
1999-11-01 12:00:00,1999.0,11.0,1.0,12:00:00,4.023350,32.0,14.0,33.560,4.60
1999-11-01 13:00:00,1999.0,11.0,1.0,13:00:00,4.828020,32.0,12.0,29.450,5.75
1999-11-01 14:00:00,1999.0,11.0,1.0,14:00:00,4.828020,32.0,12.0,29.450,5.75


In [ ]:
#studying correlation between variables
#we can also draw correlation plots (idea)
df4.iloc[:, 4:].corr(method = 'pearson')

,vsby,tmpc,dwpc,relh,sped
vsby,1.000000,0.515897,-0.148439,-0.586477,0.392256
tmpc,0.515897,1.000000,0.258315,-0.788054,0.342501
dwpc,-0.148439,0.258315,1.000000,0.324363,-0.112302
relh,-0.586477,-0.788054,0.324363,1.000000,-0.372519
sped,0.392256,0.342501,-0.112302,-0.372519,1.000000


In [ ]:
#defining classes for fog intensity, 4 - most foggy, 0 - no fog
def intensity(dfj, to_col, from_col):
    dfj.loc[:, to_col] = 9
    dfj.loc[(dfj[from_col]>=1.0),to_col]=0
    dfj.loc[(dfj[from_col]<1.0) & (dfj[from_col] >=0.5000),to_col]=1
    dfj.loc[(dfj[from_col]<0.50) & (dfj[from_col] >=0.200),to_col]=2
    dfj.loc[(dfj[from_col]<0.200) & (dfj[from_col] >=0.050),to_col] = 3
    dfj.loc[(dfj[from_col]<0.050),to_col]= 4
    return dfj

df5 = intensity(df4, "intensity", "vsby")

C:\Users\ayush\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
df5[df5.intensity != 0].head(50)

,year,month,day,time,vsby,tmpc,dwpc,relh,sped,intensity
valid,,,,,,,,,,
1999-11-02 06:00:00,1999.0,11.0,2.0,06:00:00,0.804670,21.0,15.0,68.5500,0.0,2
1999-11-02 08:00:00,1999.0,11.0,2.0,08:00:00,0.804670,24.0,16.0,60.9000,0.0,2
1999-11-07 21:00:00,1999.0,11.0,7.0,21:00:00,0.611549,22.5,8.5,40.7950,0.0,2
1999-11-07 22:00:00,1999.0,11.0,7.0,22:00:00,0.611549,21.0,8.0,43.1300,0.0,2
1999-11-07 23:00:00,1999.0,11.0,7.0,23:00:00,0.611549,21.0,7.0,40.2900,0.0,2
1999-11-08 00:00:00,1999.0,11.0,8.0,00:00:00,0.804670,21.0,7.0,40.2900,0.0,2
1999-11-20 22:00:00,1999.0,11.0,20.0,22:00:00,0.804670,19.0,8.0,48.8300,0.0,2
1999-11-20 23:00:00,1999.0,11.0,20.0,23:00:00,0.804670,18.0,8.0,51.9800,0.0,2
1999-11-21 00:00:00,1999.0,11.0,21.0,00:00:00,0.804670,17.0,7.0,51.7200,0.0,2


In [ ]:
#check % of fog events
(df5[df5.intensity!= 0].count()[0])/(df5.count()[0])

0.2550435656836461

In [ ]:
#trying Linear Regression directly
from sklearn.linear_model import LinearRegression
X = df4.iloc[:,5:-4].values
y = df4.loc[:,'vsby'].values.reshape(-1,1)

In [ ]:
X_train,  X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25, random_state = 123)

In [ ]:
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(X_train, y_train)  # perform linear regression
Y_pred = linear_regressor.predict(X_test)

In [ ]:
linear_regressor.coef_

array([[ 0.06203715, -0.04164322, -0.01919018,  0.07002297]])

In [ ]:
dfx = pd.DataFrame()
dfx["results"] = [Y_pred[i][0] for i in range(Y_pred.shape[0])]
dfx["labels"] = [y_test[i][0] for i in range(y_test.shape[0])]


In [ ]:
dfy = intensity(dfx, "rlabel","results")
dfy = intensity(dfy, "tlabel","labels")

In [ ]:
dfy1 = dfy
#confusion matrix
print("TP: ", dfy1[(dfy1.rlabel != 0)&(dfy1.tlabel != 0)].shape[0])
print("TN: ", dfy1[(dfy1.rlabel == 0)&(dfy1.tlabel == 0)].shape[0])
print("FP: ", dfy1[(dfy1.rlabel != 0)&(dfy1.tlabel == 0)].shape[0])
print("FN: ", dfy1[(dfy1.rlabel == 0)&(dfy1.tlabel != 0)].shape[0])

#precision, accuracy recall
print("Precision: ", dfy1[(dfy1.rlabel != 0)&(dfy1.tlabel != 0)].shape[0]/dfy1[(dfy1.rlabel != 0)].shape[0])
print("Recall: ", dfy1[(dfy1.rlabel != 0)&(dfy1.tlabel != 0)].shape[0]/dfy1[dfy1.tlabel!=0].shape[0])
print("Accuracy: ", dfy1[dfy1.rlabel == dfy1.tlabel].shape[0]/dfy1.shape[0])

TP:  2931
TN:  21537
FP:  801
FN:  4571
Precision:  0.7853697749196141
Recall:  0.39069581444948015
Accuracy:  0.760053619302949


In [ ]:
#defining fog event (if M out of N conscecutive entries are foggy)
def fog_event_identify(dfk):
    intensity = np.array(dfk['intensity'])
    #intensity = np.append(intensity, [0,0,0,0])
    len_intensity= len(intensity)
    fogevent = np.full(len_intensity,-6)
    N = 5
    M = 3

    i = 0
    event_no = 1
    zero_count = 0
    last_fogentry = 0
    peak = 0
    while i<len_intensity-N:
        j = i + 1
        last_fogentry = j
        peak = 0
        if intensity[i]!=0:
            zero_count = 0
            while zero_count<=N-M and j<len_intensity-1:
                if intensity[j]==0:
                    zero_count = zero_count + 1
                else:
                    last_fogentry = j
                if intensity[j]>=2:
                    peak = 1
                j = j + 1
        if j<i+N+1 or peak==0:
            i = i + 1
        else:
            fogevent[i:j-1]=intensity[i:j-1]
            fogevent[last_fogentry+1:j-1]=-6
            event_no = event_no+1
            i=j-1
    len_event_index= len(fogevent)
    event_index = np.full(len_event_index,-5)
    num=0
    for i in range(len_event_index):
        c=i+1
        if c<len_event_index:
            if fogevent[i]==-6 and fogevent[c]!=-6:
                num=num+1
                j=c
                while fogevent[j]!=-6:
                    j=j+1
                    if j>=len_event_index:
                        break
                for h in range(c,j):
                    event_index[h]=num

    dfk['fogevent'] = fogevent
    dfk['event_index'] = event_index

    return dfk

df6 = fog_event_identify(df5)
df6

In [ ]:
#if fog event then label is 1 otherwise 0
df6.loc[:, 'label'] = 0
df6.loc[df6.fogevent != -6, 'label'] = 1

C:\Users\ayush\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
df6.shape

(119360, 13)

In [ ]:
#aggregating on a daily level
#define a day as foggy if there's atleast one fog event in the day
#take average of the meteorological parameters
df7 = df6.groupby(['year', 'month','day']).agg({'label': max, 'relh': 'mean', 'sped': 'mean', 'tmpc': 'mean', 'dwpc': 'mean'}).reset_index()

In [ ]:
df7[df7.label == 1].shape

(1102, 8)

In [ ]:
#fog events year wise
#year 1999 and 2020 have data of only 2 months, so ignore
df8 =df7.groupby(['year']).agg({'label': sum})
df8

,label
year,
1999.0,13
2000.0,35
2001.0,49
2002.0,27
2003.0,37
2004.0,48
2005.0,31
2006.0,45
2007.0,45


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = df7[['relh', 'sped', 'tmpc', 'dwpc']]
y = df7[['label']]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25, random_state = 13)

In [ ]:
df7.shape

(3065, 8)

In [ ]:
lr_model = LogisticRegression().fit(X_train,y_train)

C:\Users\ayush\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
pred = lr_model.predict(X_test)
results = pd.DataFrame(pred, columns = ['rlabel'])
results.loc[:, 'tlabel'] = np.array(y_test)

In [ ]:
y_test.shape

(767, 1)

In [ ]:
y_test[y_test == 1].agg('count')

label    292
dtype: int64

In [ ]:
dfy1 = results
#confusion matrix
print("TP: ", dfy1[(dfy1.rlabel != 0)&(dfy1.tlabel != 0)].shape[0])
print("TN: ", dfy1[(dfy1.rlabel == 0)&(dfy1.tlabel == 0)].shape[0])
print("FP: ", dfy1[(dfy1.rlabel != 0)&(dfy1.tlabel == 0)].shape[0])
print("FN: ", dfy1[(dfy1.rlabel == 0)&(dfy1.tlabel != 0)].shape[0])

#precision, accuracy recall
print("Precision: ", dfy1[(dfy1.rlabel != 0)&(dfy1.tlabel != 0)].shape[0]/dfy1[(dfy1.rlabel != 0)].shape[0])
print("Recall: ", dfy1[(dfy1.rlabel != 0)&(dfy1.tlabel != 0)].shape[0]/dfy1[dfy1.tlabel!=0].shape[0])
print("Accuracy: ", dfy1[dfy1.rlabel == dfy1.tlabel].shape[0]/dfy1.shape[0])

TP:  199
TN:  411
FP:  64
FN:  93
Precision:  0.7566539923954373
Recall:  0.6815068493150684
Accuracy:  0.7953063885267275
